# Arbre de Merkle

In [ ]:
pip install typing

In [ ]:
pip install treelib

In [114]:
from hashlib import sha512
from treelib import  Tree
from typing import List

In [115]:
def hash(val: str):
    return sha512(val.encode('utf-8')).hexdigest()

class NoeudArbre:
    def __init__(self, gauche, droite, valeur: str,contenu) :
        self.gauche: NoeudArbre = gauche      # Fils de gauche
        self.droite: NoeudArbre = droite      # Fild de droite
        self.valeur = valeur                  # hash du gauche + droite
        self.contenu = contenu                # contenu du gauche + droite


In [116]:
class ArbreDeMerkle:
    def __init__(self, valeurs: List[str]) -> None:
        self.ArbreConstruct(valeurs)

    def ArbreConstruct(self, valeurs: List[str]) -> None:

        #initialiser les noeuds (transactions) de la plus basse hierarchie
        feuilles: List[NoeudArbre] = [NoeudArbre(None, None, hash(e), e) for e in valeurs]

         #rendre le nombre de transactions pair grace au slicing et on fait une copie du dernier élément pour ne pas tomber dans le piège d'avoir la meme référence pour 2 Noeuds
        if len(feuilles) % 2 == 1:
            feuilles.append(feuilles[-1:][0])    #retourner un copie du dernier NoeudArbre, avec une nouvelle référence
        self.root: NoeudArbre = self.BuildNoeudArbres(feuilles)

    def BuildNoeudArbres(self, NoeudArbres: List[NoeudArbre]) -> NoeudArbre:

        if len(NoeudArbres) % 2 != 0:      # checker si les noeuds sont impairs
            NoeudArbres.append(NoeudArbres[:-1][0])

        moitié: int = len(NoeudArbres) // 2

        if len(NoeudArbres) == 2:
            return NoeudArbre(NoeudArbres[0], NoeudArbres[1], hash(NoeudArbres[0].valeur + NoeudArbres[1].valeur), NoeudArbres[0].contenu+ " + " +NoeudArbres[1].contenu)

        gauche: NoeudArbre = self.BuildNoeudArbres(NoeudArbres[:moitié])

        droite: NoeudArbre = self.BuildNoeudArbres(NoeudArbres[moitié:])

        valeur: str = hash(gauche.valeur + droite.valeur)

        contenu: str = self.BuildNoeudArbres(NoeudArbres[:moitié]).contenu+ " + " +self.BuildNoeudArbres(NoeudArbres[moitié:]).contenu

        return NoeudArbre(gauche, droite, valeur, contenu)

    def ConceptionArbre(self) -> None:
         self.tree = Tree()
         self.tree.create_node(self.root.contenu, self.root.contenu)   # initialiser l'hierarchie par le root
         self.ConceptionChilds(self.root)  # créer les autres noeuds de l'hierarchie
         self.tree.show()

    def ConceptionChilds(self, NoeudArbre) -> None:
        if NoeudArbre != None:   #condition d'arret
            if NoeudArbre.gauche != None: # s'il y a un noeud de gauche forcément il a un noeud de droite donc on les ajoute à l'arbre
                  self.tree.create_node(NoeudArbre.gauche.contenu, NoeudArbre.gauche.contenu, parent=NoeudArbre.contenu)
                  self.tree.create_node(NoeudArbre.droite.contenu, NoeudArbre.droite.contenu, parent=NoeudArbre.contenu)

            self.ConceptionChilds(NoeudArbre.gauche)
            self.ConceptionChilds(NoeudArbre.droite)



In [117]:
elem =["trsc"+str(i) for i in range(1,8)] # génération de 7 transactions à titre d'exemple

mtree= ArbreDeMerkle(elem)

In [118]:
mtree.ConceptionArbre()

DuplicatedNodeIdError: Can't create node with ID 'trsc7'

# Visualisation de l'hierarchie de l'arbre

In [119]:
mtree.tree.show()

trsc1 + trsc2 + trsc3 + trsc4 + trsc5 + trsc6 + trsc7 + trsc7
├── trsc1 + trsc2 + trsc3 + trsc4
│   ├── trsc1 + trsc2
│   │   ├── trsc1
│   │   └── trsc2
│   └── trsc3 + trsc4
│       ├── trsc3
│       └── trsc4
└── trsc5 + trsc6 + trsc7 + trsc7
    ├── trsc5 + trsc6
    │   ├── trsc5
    │   └── trsc6
    └── trsc7 + trsc7
        └── trsc7

